In [1]:
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
import random
from diffusers import DDIMScheduler
import numpy as np
from Freeprompt.diffuser_utils import FreePromptPipeline
from Freeprompt.freeprompt_utils import register_attention_control_new
from torchvision.utils import save_image
from torchvision.io import read_image
from Freeprompt.freeprompt import SelfAttentionControlEdit,AttentionStore


#### Model Construction

In [3]:
# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "runwayml/stable-diffusion-v1-5"

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = FreePromptPipeline.from_pretrained(model_path, scheduler=scheduler).to(device)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/apsarapangu/disk1/wushi.lby/anaconda3/envs/ldm/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/apsarapangu/disk1/wushi.lby/anaconda3/envs/ldm/lib/python3.8/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:115: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.16.1",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "sample_max_value": 1.0,
  "set_alpha_t

#### Fake image editing

In [4]:

# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def seed_everything(seed: Optional[int] = None, workers: bool = False) -> int:

    if seed is None:
        seed = os.environ.get("PL_GLOBAL_SEED")
    seed = int(seed)

    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    os.environ["PL_SEED_WORKERS"] = f"{int(workers)}"

    return seed

seed = 42
seed_everything(seed)

self_replace_steps = .8
NUM_DIFFUSION_STEPS = 50

out_dir = "examples/outputs"


source_prompt = 'a photo of green ducks walking on street'
target_prompt = 'a photo of rubber ducks walking on street'

start_code = torch.randn([1, 4, 64, 64], device=device)
start_code = start_code.expand(2, -1, -1, -1)

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    )

save_image(results[0], os.path.join(out_dir, str(source_prompt)+'.jpg'))
save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))


/apsarapangu/disk1/wushi.lby/image_edit/sup/code/Free_prompt_editing/Freeprompt/diffuser_utils.py:134: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  latents_shape = (batch_size, self.unet.in_channels, height//8, width//8)
DDIM Sampler: 100%|██████████| 50/50 [00:10<00:00,  4.68it/s]
